In [62]:
# Import libraries
import openai, jsonlines
import pandas as pd

In [63]:
# Setup OpenAI API
secret_api_key="sk-UgVTxJkXAOwlfRIJbh5t69Vl9O6ifNLXXJR941F5"
openai.api_key = secret_api_key

In [64]:
# Test
response = openai.Completion.create(engine="davinci", prompt="This is a test", max_tokens=5)
print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": ". Please do not join"
    }
  ],
  "created": 1620356838,
  "id": "cmpl-2wl7eD7HuZblrR9rHPI5SQyLetFak",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


In [49]:
# Load data
df = pd.read_csv("Data/text-categorisation-data.csv")
df.head()

,id,rating,comments,comment category
0,1,Good,AMAZING! best scanning experience ever!,NaN
1,2,Good,"A bit slow but its working, also got the day o...",NaN
2,3,Good,NaN,NaN
3,4,Good,NaN,NaN
4,5,Good,NaN,NaN


In [11]:
# Group by category
df.groupby(by="comment category").count().sort_values(by='id', ascending=False).head()

,id,rating,comments
comment category,,,
Inaccurate scan,332,332,325
Take too long,108,108,106
No valuable reason,66,66,63
Full screen/ no frame,61,61,59
Difficult to use,59,59,59


In [57]:
# Sample from two categories
n_samples = 10

def sample_category(df, category, n=10):
    df = df\
        .copy()\
        .loc[df["comment category"].isin([category]),["comments","comment category"]]\
        .rename(columns={"comments":"text","comment category":"label"})\
        .sample(n)
    return df

sample_df = pd.concat([sample_category(df=df,category="Inaccurate scan",n=n_samples),
                       sample_category(df=df,category="Take too long",n=n_samples)])
sample_l = sample_df.to_dict('records')
print(len(sample_l))

20


In [60]:
# Export json lines file
with jsonlines.open("Data/text-categorisation-data.jsonl", mode='w') as writer:
    for d in dict_l:
        writer.write(d)

In [66]:
# Upload labelled examples to OpenAI
openai.File.create(file=open("Data/text-categorisation-data.jsonl"), purpose="classifications")


Upload progress:   0%|          | 0.00/35.9k [00:00<?, ?it/s]
Upload progress:  23%|██▎       | 8.19k/35.9k [00:03<00:12, 2.26kit/s]
Upload progress:  46%|████▌     | 16.4k/35.9k [00:03<00:04, 4.51kit/s]
Upload progress:  68%|██████▊   | 24.6k/35.9k [00:03<00:01, 6.76kit/s]
Upload progress: 100%|██████████| 35.9k/35.9k [00:03<00:00, 9.86kit/s]


<File file id=file-MICfYopkuLi28oBR8IWQDazX at 0x7fd08c6962f0> JSON: {
  "bytes": 35620,
  "created_at": 1620356886,
  "filename": "Data/text-categorisation-data.jsonl",
  "id": "file-MICfYopkuLi28oBR8IWQDazX",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}

In [67]:
# Query classifications
def query_classifications(query):
    clf_obj = openai.Classification.create(
        #file="file-6zkkNvBUTAWRNzwSayFc3LUs", # Trained on all labels
        file="file-MICfYopkuLi28oBR8IWQDazX", # Trained on 10 random labels
        query=query,
        search_model="ada", 
        model="curie", 
        max_examples=3
    )
    return clf_obj

In [69]:
# Test query classifications
q1 = "The scan took way too long!!!"
q2 = "I couldn't detect my driver's license!!"
q3 = "The scan got my name wrong"
q4 = "I waited 5 mins for the scan and just gave up on it in the end..."

clf1 = query_classifications(q1)
clf2 = query_classifications(q2)
clf3 = query_classifications(q3)
clf4 = query_classifications(q4)

In [73]:
clf4

<OpenAIObject classification at 0x7fd08c6a3ef0> JSON: {
  "completion": "cmpl-2wlGz7COMwUbkbP8dJuCfyuFWRxdR",
  "file": "file-MICfYopkuLi28oBR8IWQDazX",
  "label": "Take too long",
  "model": "curie:2020-05-03",
  "object": "classification",
  "search_model": "ada:2020-05-03",
  "selected_examples": [
    {
      "document": 1,
      "label": "Inaccurate scan",
      "object": "search_result",
      "score": 51.966,
      "text": "Just did not scan correctly."
    },
    {
      "document": 2,
      "label": "Take too long",
      "object": "search_result",
      "score": 42.453,
      "text": "Just took ages to scan "
    },
    {
      "document": 0,
      "label": "Take too long",
      "object": "search_result",
      "score": 95.939,
      "text": "took about 5 mins to scan, just give me the option to put the details in manually "
    }
  ]
}